# *Computing a MFW coronal model*
***

The magnetofrictional wind (MFW) model computes the coronal magnetic field by assuming

\begin{eqnarray}
\mathbf{B} = f \nabla u
\end{eqnarray}

where the scalar function $f = f(r)$ is related to a given solar wind radial outflow speed $\mathbf{v}_\mathrm{sw} = v_r(r) \, \mathbf{e}_r$ by

\begin{eqnarray}
v_r = \frac{1}{\nu_0} \frac{1}{f} \frac{df}{dr}
\end{eqnarray}

where $\nu_0$ is a constant. The resulting magnetic field is an equilibirum solution of Faraday's law, i.e. $\mathbf{v} \times \mathbf{B} = 0$ where the total flow velocity is given by $\mathbf{v} = \mathbf{v}_\mathrm{MF} + \mathbf{v}_\mathrm{sw}$ with $\mathbf{v}_\mathrm{MF}$ the magnetofriction velocity

\begin{eqnarray}
\mathbf{v}_\mathrm{MF} = \frac{1}{\nu_0} \frac{\mu_0 \mathbf{J} \times \mathbf{B}}{B^2} 
\end{eqnarray}

Thus, in the low corona where $v_\mathrm{sw}$ is small, a nearly force-free magnetic field is obtained, whereas in the upper corona the solution is dominated by the outflow. Unlike the PFSS model, the magnetic field carries a current throughout the domain, and is given by

\begin{eqnarray}
\mu_0 \mathbf{J} = \frac{1}{f} (\nabla f) \times \mathbf{B}
\end{eqnarray}

Thus, with $f=f(r)$ the radial current $J_r = 0$ throughout.

In [ ]:
import numpy as np
import scipy.constants as constants
import matplotlib.pyplot as plt

In [ ]:
import sunpy

In [ ]:
import pysmsh
import cider

### Load magnetogram & process

Load a HMI synchronic magnetogram

In [ ]:
import cider.magnetogram.hmi

In [ ]:
magnetogram_file = "../data/hmi.mrdailysynframe_polfil_720s.20211009_120000_TAI.Mr_polfil.fits"

In [ ]:
raw_magnetogram = cider.magnetogram.hmi.read_hmi_daily_synframe(magnetogram_file)

Remap to a uniform lon-lat grid which is required by the solver

In [ ]:
import cider.utils.map
import cider.magnetogram.flux

In [ ]:
# Create an empty map with the requested resolution
uniform_map \
    = cider.utils.map.create_full_sun_plate_carree_map(raw_magnetogram,
                                                       deg_per_pixel=1.0,
                                                       frame=raw_magnetogram.coordinate_frame.name)

In [ ]:
remapped_magnetogram = cider.utils.map.regrid_to_grid_of_map(raw_magnetogram, uniform_map)

Balance the magnetogram

In [ ]:
balanced_magnetogram = cider.magnetogram.flux.Balance().multiplicative(remapped_magnetogram)

In [ ]:
balanced_magnetogram.peek(vmin=-50, vmax=50)

### Define the outflow profile

The form of the outflow dictates the magnetic field topology.

Here we implement a simple flow profile such that

\begin{eqnarray}
v_r = \frac{1}{2}(v_1 + v_0) + \frac{1}{2}(v_1 - v_0) \tanh((r-r_1)/w)
\end{eqnarray}

so that $v_r \to v_1$ when $r \to \infty$ and $v_r(r_1) = \frac{1}{2}(v_1 + v_0)$ at some radius $r_1$ and $v_0$ can be chosen such that the wind speed is low in the low corona.

Note that in this model it is the product $\nu_0 v_r$ that determines the wind forcing, so the effect of the wind can be additionally adjusted by varying the magnetofrictional coefficient.

In [ ]:
class HyperbolicTanFlowProfile:
    
    def __init__(self):
        
        # MF coefficient
        # Chosen in this particular case so that the open flux is close (within a few percent) to that
        # given by a PFSS model with the same magnetogram with a source surface radius of 2.5 RSun
        self.nu0 = 4.5e-14

        # Terminal wind speed
        self.v1 = 500.0e3
        
        # Width of transition
        self.w = 2.0*sunpy.sun.constants.radius.value
        
        # Radius at which the average speed is obtain
        # In practice, if v0 is ~0, this is close to v1/2
        self.r1 = 4.0*sunpy.sun.constants.radius.value
         
        # Compute wind speed shift in lower corona
        # based on specifying a target wind speed at r=RSun
        v_at_boundary = 1e3
        
        tanh = np.tanh((sunpy.sun.constants.radius.value - self.r1)/self.w)
        
        self.v0 = (2.0*v_at_boundary - self.v1*(1.0 + tanh))/(1.0 - tanh)
                
    def f(self, r):

        log_tanh_term = np.log(np.tanh((r-self.r1)/self.w) + 1.0)

        integral_v_dr = 0.5*r*(self.v1 + self.v0) + 0.5*(self.v1 - self.v0)*(r - self.w*log_tanh_term)
   
        return np.exp(self.nu0*integral_v_dr)

    def vr(self, r):
        
        return 0.5*(self.v1 + self.v0) + 0.5*(self.v1 - self.v0)*np.tanh((r-self.r1)/self.w)
        

In [ ]:
flow = HyperbolicTanFlowProfile()

In [ ]:
# Compute vr using a simple finite difference as a check that vr is consistent with f
rvalues = np.linspace(1.0, 15.0, 1024)*sunpy.sun.constants.radius.value
vr = np.zeros(len(rvalues))

for i, r in enumerate(rvalues):

    dr = 1.0
    dfdr = (flow.f(r+dr) - flow.f(r-dr))/(2.0*dr)
    
    vr[i] = (1.0/flow.nu0)*(1.0/flow.f(r))*dfdr

plt.plot(rvalues/sunpy.sun.constants.radius.value, flow.vr(rvalues)/1e3, alpha=0.5, color='r', lw=3)
plt.plot(rvalues/sunpy.sun.constants.radius.value, vr/1e3, color='k')

plt.xlabel(r"$r$ [solar radii]")
plt.ylabel(r"$v_r$ [km / s]");

### Compute the model

In [ ]:
import cider.models.mfw

Specify the radial grid coordinates. Unlike the PFSS model, the outer boundary can be extended to the upper corona.

In [ ]:
r = np.linspace(1.0, 10.0, 2*256)*sunpy.sun.constants.radius.value

In [ ]:
# Instantiate the model
mfw = cider.models.mfw.MagnetofrictionalWindModel(balanced_magnetogram, r, flow)

In [ ]:
# Compute the solution
mfw.compute()

In [ ]:
# Compute the magnetic field
magnetic_field = mfw.magnetic_field()

Do some additional analysis: compute the divergence of the magnetic field

In [ ]:
import pysmsh.difference.staggered_curvilinear as curvilinear

In [ ]:
div_magnetic_field = pysmsh.Field.Scalar(mfw.mesh, "cell_centered")

In [ ]:
curvilinear.divergence(magnetic_field, div_magnetic_field, curvilinear.SphericalGeometry())

Compute the current density

In [ ]:
current_density = pysmsh.Field.Vector(mfw.mesh, "edge_staggered")

In [ ]:
curvilinear.curl(magnetic_field, current_density, curvilinear.SphericalGeometry())

In [ ]:
# Need to scale by 1/mu_0
for d in (0, 1, 2):
    current_density.data[d][:, :, :] *= 1.0/constants.mu_0

Output the cell-centered magnetic field components for external visualization

In [ ]:
import pyevtk.hl as evtk

In [ ]:
B = magnetic_field

evtk.gridToVTK("mfw",
               B.mesh.edges.r/sunpy.sun.constants.radius.value,
               B.mesh.edges.clt,
               B.mesh.edges.lon,
               cellData={"Br" : 0.5*(B.data[0][1::, :, :] + B.data[0][0:-1, :, :]),
                         "Bt" : 0.5*(B.data[1][:, 1::, :] + B.data[1][:, 0:-1, :]),
                         "Bp" : 0.5*(B.data[2][:, :, 1::] + B.data[2][:, :, 0:-1]),
                         "divB" : div_magnetic_field.data[:, :, :]
                        }
              )

Compute flux balance as a function of radius

In [ ]:
B = magnetic_field

signed_flux = np.zeros(len(B.mesh.edges.r))
unsigned_flux = np.zeros(len(B.mesh.edges.r))

for i, r in enumerate(B.mesh.edges.r):

    signed_flux[i], unsigned_flux[i], area \
        = cider.magnetogram.flux.signed_unsigned_flux_kernel(B.data[0][i, :, :],
                                                             B.mesh.edges.clt,
                                                             B.mesh.edges.lon,
                                                             r)

In [ ]:
pfss_value = 2.24e+18 # Computed elsewhere

plt.plot(B.mesh.edges.r/sunpy.sun.constants.radius.value, unsigned_flux/pfss_value)
plt.xlim(2.0, 10)
plt.ylim(0.95, 1.2)